In [28]:
#import lib
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import cv2
import os,glob,shutil
import PIL
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import pathlib

In [53]:
input_dir = './Images_2'
global_dir = input_dir + '/*.png'
#to locate all files in directory
temp = glob.glob(global_dir)
#print num of images in that folder
print(len(temp))

300


In [56]:
x = temp
#read and resize images from selected file
images = np.array([cv2.resize(cv2.imread(file),(128,128)) for file in x], dtype=np.float32)
images /=30.0


In [57]:


#load mobilenetv2 model pre-trained on internt
model = tf.keras.applications.MobileNetV2(input_shape=(128,128,3), include_top=False, weights=('imagenet'))

#get image embeddings using the model
embeddings = model.predict(images) #images is a numpy array of shape (n_images,128,128,3)

#reshape embeddings to have one dimension per image
embeddings = embeddings.reshape(embeddings.shape[0], -1)
#embeddings is a numpy array of shape (n_images,embedddings_size)
embeddings.shape

10/10 [==============================] - 4s 273ms/step


(300, 20480)

In [58]:
n_clusters = 20

#fit kmeans model to the image embeddinsg

kmeans = KMeans(n_clusters=n_clusters,verbose=0,random_state=42)
kmeans.fit(embeddings)

#make predictions using the fitted model
cluster_labels = kmeans.predict(embeddings)

C:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [59]:
for i in range(n_clusters):
    os.makedirs("cluster"+str(i),exist_ok=True)

#copy images to corresponding cluster directories
for path, cluster_label in zip(temp, cluster_labels):
    shutil.copy2(os.fsdecode(path), "cluster"+str(cluster_label))



In [60]:
k=20
for i in range (k):
    cluster_path="cluster"+str(i)+'/*.png'
    images=[cv2.imread(img_path,0) for img_path in glob.glob(cluster_path)]